In [0]:
%sql
select current_catalog(), current_schema()

In [0]:
%sql
create catalog if not exists ecommerce;
use catalog ecommerce;
create schema if not exists v01;
use schema v01;
create volume if not exists raw;


In [0]:
df = (spark
      .read
      .format("csv")
      .option("header", "true")
      .option("inferSchema", "true")
      .load('/Volumes/workspace/default/adventure_works/AdventureWorks_Customers.csv'))

In [0]:
display(df)

In [0]:
(df
 .write
 .format("parquet")
 .save("/Volumes/ecommerce/v01/raw/AdventureWorks_Customers"))

In [0]:
%sql
select * 
from parquet.`/Volumes/ecommerce/v01/raw/AdventureWorks_Customers/`;

In [0]:
%sql
--DROP TABLE IF EXISTS historical_users_bronze_ctas_rf
DROP TABLE IF EXISTS adventureWorks_customers_bronze_ctas_rf;

--CREATE THE DELTA TABLE 
CREATE TABLE adventureWorks_customers_bronze_ctas_rf
SELECT * 
FROM read_files('/Volumes/ecommerce/v01/raw/AdventureWorks_Customers/',
format => 'parquet')
LIMIT 10;

--PREVIEW THE DELTA TABLE 
SELECT *
FROM adventureWorks_customers_bronze_ctas_rf
LIMIT 10; 



In [0]:
%sql
DESCRIBE TABLE EXTENDED ecommerce.v01.adventureworks_customers_bronze_ctas_rf

In [0]:
%sql
SHOW columns in ecommerce.v01.adventureworks_customers_bronze_ctas_rf

In [0]:
%sql
DROP TABLE ecommerce.v01.adventureworks_customers_bronze_ci purge 

In [0]:
%sql
--Create an empty table 
CREATE TABLE IF NOT EXISTS ecommerce.v01.adventureworks_customers_bronze_ci(
CustomerKey integer,
Prefix string,
FirstName string,
LastName string,
BirthDate date,
MaritalStatus string,
Gender string,
EmailAddress string,
AnnualIncome string
)

In [0]:
%sql
COPY INTO  ecommerce.v01.adventureworks_customers_bronze_ci
FROM '/Volumes/ecommerce/v01/raw/AdventureWorks_Customers/'
FILEFORMAT = parquet 
COPY_OPTIONS('mergeSchema' = 'true' )

In [0]:
%sql
SELECT * FROM ecommerce.v01.adventureworks_customers_bronze_ci